In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneOut, cross_val_score
from scipy.optimize import differential_evolution
import os

# -----------------------------
# Taguchi L16 dataset
# -----------------------------
data = [
    [0,0,0,10,100,54.1,5.13],
    [0,10,3,12,100,65.34,13.8],
    [0,15,5,14,120,70.23,17.8],
    [0,20,8,16,120,70.09,16.46],
    [10,0,3,14,120,61.2,7.12],
    [10,10,0,16,120,73.22,18.83],
    [10,15,8,10,100,82.5,28.43],
    [10,20,5,12,100,79.42,24.98],
    [15,0,5,16,100,67.5,19.87],
    [15,10,8,14,100,79.85,26.12],
    [15,15,0,12,120,74.45,22.13],
    [15,20,3,10,120,78.4,26.2],
    [20,0,8,12,120,71.2,18.83],
    [20,10,5,10,120,82.4,27.75],
    [20,15,3,16,100,82.98,28.06],
    [20,20,0,14,100,77.6,22.83],
]
df = pd.DataFrame(data, columns=["S","B","N","P","T","FS","IS"])

# -----------------------------
# Function to fit Ridge with CV
# -----------------------------
def fit_best_ridge(X, y, degrees=[1,2], alphas=np.logspace(-3, 3, 20), name=""):
    loo = LeaveOneOut()
    best_r2 = -np.inf
    best_model = None
    best_poly = None
    best_scaler = None
    best_degree = None
    best_alpha = None

    for deg in degrees:
        poly = PolynomialFeatures(degree=deg, include_bias=False)
        X_poly = poly.fit_transform(X)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_poly)

        for alpha in alphas:
            ridge = Ridge(alpha=alpha)
            try:
                r2_cv = cross_val_score(ridge, X_scaled, y, cv=loo, scoring='r2').mean()
            except Exception:
                r2_cv = -np.inf
            if np.isfinite(r2_cv) and r2_cv > best_r2:
                best_r2 = r2_cv
                best_model = ridge
                best_poly = poly
                best_scaler = scaler
                best_degree = deg
                best_alpha = alpha

    if best_model is None:
        print(f"⚠️ Warning: No valid model found for {name}. Using default Ridge(alpha=1.0, degree=1)")
        best_model = Ridge(alpha=1.0)
        best_poly = PolynomialFeatures(degree=1, include_bias=False)
        best_scaler = StandardScaler()
        X_scaled = best_scaler.fit_transform(best_poly.fit_transform(X))
        best_model.fit(X_scaled, y)
        r2_train = r2_score(y, best_model.predict(X_scaled))
        return best_model, best_poly, best_scaler, 1, 1.0, r2_train, np.nan

    # Fit best model on full data
    X_scaled = best_scaler.fit_transform(best_poly.fit_transform(X))
    best_model.fit(X_scaled, y)
    r2_train = r2_score(y, best_model.predict(X_scaled))

    print(f"\nBest Ridge model for {name}:")
    print(f" Degree = {best_degree},  Alpha = {best_alpha:.3e}")
    print(f" R² (Train) = {r2_train:.4f},  R² (LOO-CV) = {best_r2:.4f}")

    return best_model, best_poly, best_scaler, best_degree, best_alpha, r2_train, best_r2


# -----------------------------
# Fit models for FS and IS
# -----------------------------
X = df[["S","B","N","P","T"]].values
y_FS = df["FS"].values
y_IS = df["IS"].values

ridge_fs, poly_fs, scaler_fs, deg_fs, alpha_fs, r2_train_fs, r2_cv_fs = fit_best_ridge(X, y_FS, name="Flexural Strength")
ridge_is, poly_is, scaler_is, deg_is, alpha_is, r2_train_is, r2_cv_is = fit_best_ridge(X, y_IS, name="Impact Strength")

# -----------------------------
# Predictions and error %
# -----------------------------
df["FS_pred"] = ridge_fs.predict(scaler_fs.transform(poly_fs.transform(X)))
df["IS_pred"] = ridge_is.predict(scaler_is.transform(poly_is.transform(X)))
df["FS_err%"] = np.abs((df["FS_pred"] - df["FS"]) / df["FS"]) * 100
df["IS_err%"] = np.abs((df["IS_pred"] - df["IS"]) / df["IS"]) * 100

print("\nPrediction summary:")
print(df[["S","B","N","P","T","FS","FS_pred","FS_err%","IS","IS_pred","IS_err%"]].head())

# -----------------------------
# Differential Evolution Optimization
# -----------------------------
def taguchi_predict(model, poly, scaler, x):
    x = np.array(x).reshape(1, -1)
    x_poly = poly.transform(x)
    x_scaled = scaler.transform(x_poly)
    return model.predict(x_scaled)[0]

fs_min, fs_max = y_FS.min(), y_FS.max()
is_min, is_max = y_IS.min(), y_IS.max()

def combined_score(x):
    fs = taguchi_predict(ridge_fs, poly_fs, scaler_fs, x)
    isv = taguchi_predict(ridge_is, poly_is, scaler_is, x)
    fs_norm = (fs - fs_min) / (fs_max - fs_min)
    is_norm = (isv - is_min) / (is_max - is_min)
    return fs_norm + is_norm

def de_objective(x):
    return -combined_score(x)  # minimize negative score

bounds = [(0,20),(0,20),(0,8),(10,16),(100,120)]
result = differential_evolution(de_objective, bounds, maxiter=200, popsize=20, tol=1e-6, seed=42)
x_opt = result.x
best_score = -result.fun

fs_opt = taguchi_predict(ridge_fs, poly_fs, scaler_fs, x_opt)
is_opt = taguchi_predict(ridge_is, poly_is, scaler_is, x_opt)

# -----------------------------
# Identify optimum from dataset too (for reference)
# -----------------------------
score = (df["FS_pred"] / df["FS_pred"].max()) + (df["IS_pred"] / df["IS_pred"].max())
opt_idx = score.idxmax()
opt = df.loc[opt_idx]

# -----------------------------
# Save results to CSV (append mode)
# -----------------------------
opt_summary = pd.DataFrame({
    "S": [float(x_opt[0])],
    "B": [float(x_opt[1])],
    "N": [float(x_opt[2])],
    "P": [float(x_opt[3])],
    "T": [float(x_opt[4])],
    "Predicted_FS(MPa)": [float(fs_opt)],
    "Predicted_IS(kJ/m²)": [float(is_opt)],
    "Combined_Normalized_Score": [float(best_score)],
    "Best_Degree_FS": [deg_fs],
    "Alpha_FS": [alpha_fs],
    "R2_Train_FS": [r2_train_fs],
    "R2_CV_FS": [r2_cv_fs],
    "Best_Degree_IS": [deg_is],
    "Alpha_IS": [alpha_is],
    "R2_Train_IS": [r2_train_is],
    "R2_CV_IS": [r2_cv_is]
})

file_name = "optimized_result.csv"
if os.path.exists(file_name):
    opt_summary.to_csv(file_name, mode='a', header=False, index=False)
else:
    opt_summary.to_csv(file_name, index=False)

# -----------------------------
# Print Final Results
# -----------------------------
print("\n=== Optimized Result (Saved / Appended to 'optimized_result.csv') ===")
print(f"Optimum Control Factors:")
print(f"  S = {float(x_opt[0]):.2f}")
print(f"  B = {float(x_opt[1]):.2f}")
print(f"  N = {float(x_opt[2]):.2f}")
print(f"  P = {float(x_opt[3]):.2f}")
print(f"  T = {float(x_opt[4]):.2f}")

print(f"\nPredicted Properties:")
print(f"  Flexural Strength (FS) = {float(fs_opt):.2f} MPa")
print(f"  Impact Strength (IS)   = {float(is_opt):.2f} kJ/m²")

print(f"\nCombined Normalized Score = {float(best_score):.4f}")
print("\n✅ Result exported successfully to 'optimized_result.csv'")


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, Unde

⚠️ Warning: No valid model found for Flexural Strength. Using default Ridge(alpha=1.0, degree=1)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, Unde

⚠️ Warning: No valid model found for Impact Strength. Using default Ridge(alpha=1.0, degree=1)

Prediction summary:
    S   B  N   P    T     FS  FS_pred  FS_err%     IS  IS_pred  IS_err%
0   0   0  0  10  100  54.10   56.720    4.843   5.13    7.155   39.479
1   0  10  3  12  100  65.34   65.170    0.260  13.80   13.843    0.313
2   0  15  5  14  120  70.23   68.673    2.217  17.80   15.650   12.078
3   0  20  8  16  120  70.09   73.881    5.409  16.46   19.800   20.293
4  10   0  3  14  120  61.20   63.704    4.091   7.12   12.143   70.541

=== Optimized Result (Saved / Appended to 'optimized_result.csv') ===
Optimum Control Factors:
  S = 20.00
  B = 20.00
  N = 8.00
  P = 10.00
  T = 100.00

Predicted Properties:
  Flexural Strength (FS) = 88.23 MPa
  Impact Strength (IS)   = 33.73 kJ/m²

Combined Normalized Score = 2.4093

✅ Result exported successfully to 'optimized_result.csv'
